In [4]:
import pandas as pd
import numpy as np
import similaritymeasures
from io import StringIO
from frechetdist import frdist

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
field_names=['RealData','MAS', 'Raw','RMSE-MAS', 'RMSE-Raw']

def read_data(location):
    with open(location) as f:
        data = ''
        for line in f.readlines():
            if line.count(';') == 2:
                line = line + ';'
            data += line
        data = data.replace('\n', '').replace(';', ';\n')

    # skip header, go to first data block
    data = data[data.find('['):]
    curated_data = ''
    field_nr = 0
    for line in data.split('\n'):
        if len(line) == 0:
            continue

        field_nr += 1
        if field_nr == len(field_names):
            curated_data += line.replace(';', '\n')
            field_nr = 0
        else:
            curated_data += line
    
    df=pd.read_csv(StringIO(curated_data), sep=';', header=0, names=field_names)
    return df

In [111]:
data = read_data('mc_surrogate')

In [109]:
data.head()

,RealData,MAS,Raw,RMSE-MAS,RMSE-Raw
0,"[61.59, 62.08, 62.93, 64.03, 65.09, 66.24, 67....",[ 67.43530954 69.0788475 70.58806974 71.8...,[ 88.01138937 88.96768648 89.93880528 90.52...,30.087345,16.254265
1,"[61.59, 62.08, 62.93, 64.03, 65.09, 66.24, 67....",[ 88.28997728 89.4197747 90.46949917 91.3...,[ 92.48595719 93.35877978 94.1603509 94.76...,14.176917,19.234728
2,"[61.59, 62.08, 62.93, 64.03, 65.09, 66.24, 67....",[ 81.66762262 82.5870911 83.42292078 84.1...,[127.90154866 128.61439844 129.33323332 129.70...,19.050827,32.984552
3,"[61.59, 62.08, 62.93, 64.03, 65.09, 66.24, 67....",[ 94.30858334 95.25315913 96.21211492 96.7...,[117.09552502 118.00431819 118.9230905 119.48...,19.710247,32.496989
4,"[61.59, 62.08, 62.93, 64.03, 65.09, 66.24, 67....",[ 67.42900582 68.7169605 69.95169473 70.8...,[ 79.79596925 80.79071831 81.72550241 82.49...,10.345084,11.746823


In [112]:
# Convert to float arrays
field_name = 'RealData'
for i, row in enumerate(data[field_name]):
    row = row.replace('[', '').replace(']', '')
    data[field_name][i] = [float(x) for x in row.split(',')]

In [113]:
field_name = 'MAS'
for i, row in enumerate(data[field_name]):
    row = row.replace('[', '').replace(']', '')
    data[field_name][i] = [float(x) for x in row.split()]

In [115]:
field_name = 'Raw'
for i, row in enumerate(data[field_name]):
    row = row.replace('[', '').replace(']', '')
    data[field_name][i] = [float(x) for x in row.split()]

In [116]:
field_name = 'RMSE-MAS'
for i, row in enumerate(data[field_name]):
    data[field_name][i] = float(row)

In [117]:
field_name = 'RMSE-Raw'
for i, row in enumerate(data[field_name]):
    data[field_name][i] = float(row)

In [122]:
mas = data['MAS']
raw = data['Raw']
real = data['RealData']

In [124]:
frechet_raw = []
frechet_mas = []
for i in range(len(real)):
    df_mas = similaritymeasures.frechet_dist(real[i], mas[i])
    frechet_mas.append(df_mas)
    df_raw = similaritymeasures.frechet_dist(real[i], raw[i])
    frechet_raw.append(df_raw)
    
print(np.mean(frechet_raw), np.mean(frechet_mas))

38.559034268000005 25.86694442638462


In [125]:
print(np.std(frechet_raw), np.std(frechet_mas))

25.262197157853574 16.195601658878626


In [127]:
mas_rmse = data['RMSE-MAS']
raw_rmse = data['RMSE-Raw']
print(np.mean(raw_rmse), np.mean(mas_rmse))
print(np.std(raw_rmse), np.std(mas_rmse))

28.85547048223189 20.353164821836813
16.241629801588182 9.360090711700261


In [7]:
data_cma = read_data('cma_surrogate')

In [8]:
data_cma.head()

,RealData,MAS,Raw,RMSE-MAS,RMSE-Raw
0,"[61.59, 62.08, 62.93, 64.03, 65.09, 66.24, 67....",[ 80.42447821 81.57838506 82.64092455 83.5...,[ 70.02322031 70.47100842 70.94148842 71.70...,9.680872,7.947330
1,"[61.59, 62.08, 62.93, 64.03, 65.09, 66.24, 67....",[ 80.49481368 81.58067959 82.62201808 83.5...,[ 86.47309629 86.97468151 87.47651039 88.44...,9.804457,18.585057
2,"[61.59, 62.08, 62.93, 64.03, 65.09, 66.24, 67....",[ 80.5959425 81.70679455 82.72095036 83.5...,[ 73.22946392 73.61831678 74.03838602 74.67...,9.818624,10.480254
3,"[61.59, 62.08, 62.93, 64.03, 65.09, 66.24, 67....",[ 80.61144333 81.76032432 82.81870788 83.6...,[ 68.69842596 68.82984406 69.10582313 69.46...,9.734563,6.493252
4,"[61.59, 62.08, 62.93, 64.03, 65.09, 66.24, 67....",[ 80.4453563 81.56604177 82.61743645 83.4...,[ 68.76355447 69.15252579 69.57043191 70.24...,9.628156,9.000566


In [9]:
print(len(data_cma))

164


In [11]:
data_cma = data_cma[:-34]
print(len(data_cma))

130


In [13]:
# Convert to float arrays
field_name = 'RealData'
for i, row in enumerate(data_cma[field_name]):
    row = row.replace('[', '').replace(']', '')
    data_cma[field_name][i] = [float(x) for x in row.split(',')]

In [14]:
field_name = 'MAS'
for i, row in enumerate(data_cma[field_name]):
    row = row.replace('[', '').replace(']', '')
    data_cma[field_name][i] = [float(x) for x in row.split()]

In [15]:
field_name = 'Raw'
for i, row in enumerate(data_cma[field_name]):
    row = row.replace('[', '').replace(']', '')
    data_cma[field_name][i] = [float(x) for x in row.split()]

In [16]:
field_name = 'RMSE-MAS'
for i, row in enumerate(data_cma[field_name]):
    data_cma[field_name][i] = float(row)

In [17]:
field_name = 'RMSE-Raw'
for i, row in enumerate(data_cma[field_name]):
    data_cma[field_name][i] = float(row)

In [18]:
mas2 = data_cma['MAS']
raw2 = data_cma['Raw']
real2 = data_cma['RealData']

In [20]:
frechet_raw = []
frechet_mas = []
for i in range(len(real2)):
    df_mas = similaritymeasures.frechet_dist(real2[i], mas2[i])
    frechet_mas.append(df_mas)
    df_raw = similaritymeasures.frechet_dist(real2[i], raw2[i])
    frechet_raw.append(df_raw)
    
print(np.mean(frechet_raw), np.mean(frechet_mas))

35.57373479046155 19.00974428907692


In [21]:
mas_rmse2 = data_cma['RMSE-MAS']
raw_rmse2 = data_cma['RMSE-Raw']
print(np.mean(raw_rmse2), np.mean(mas_rmse2))
print(np.std(raw_rmse2), np.std(mas_rmse2))

16.807009384992714 10.074636724182879
9.76502385616441 1.1279065980188026


In [22]:
print(np.std(frechet_raw), np.std(frechet_mas))

18.23947756941678 1.4967337880417257
